In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense 
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical 
import time
import warnings
from keras.layers import Dropout, Activation, Flatten, Convolution1D, Dropout
from keras.layers import BatchNormalization
import datetime as dt
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import seaborn as sn

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
!nvidia-smi

Fri Oct 18 17:31:50 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   51C    P0             11W /   60W |     330MiB /   4096MiB |     21%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
df = pd.read_csv('preprocessed_DNN.csv', low_memory=False)

In [5]:
df["Attack_type"].value_counts()

Attack_type
Normal                   1363998
DDoS_UDP                  121567
DDoS_ICMP                  67939
SQL_injection              50826
DDoS_TCP                   50062
Vulnerability_scanner      50026
Password                   49933
DDoS_HTTP                  48544
Uploading                  36807
Backdoor                   24026
Port_Scanning              19977
XSS                        15066
Ransomware                  9689
Fingerprinting               853
MITM                         358
Name: count, dtype: int64

In [6]:
# Step 1: Filter the normal data
normal_data = df[df['Attack_type'] == 'Normal']

# Step 2: Sample 50% of the normal data
normal_data_sampled = normal_data.sample(frac=1, random_state=42)

# Step 3: Filter out only the DDoS attack types (UDP, ICMP, TCP, HTTP)
ddos_data = df[df['Attack_type'].isin(['DDoS_UDP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_HTTP'])]

# Step 4: Recombine the sampled normal data with the DDoS attack data
df = pd.concat([normal_data_sampled, ddos_data], ignore_index=True)

# Step 5: Verify the distribution of attack types
print(df["Attack_type"].value_counts())


Attack_type
Normal       1363998
DDoS_UDP      121567
DDoS_ICMP      67939
DDoS_TCP       50062
DDoS_HTTP      48544
Name: count, dtype: int64


In [7]:
feat_cols = list(df.columns)
label_col = "Attack_type"
feat_cols.remove(label_col)
print(len(feat_cols))
feat_cols

96


['arp.opcode',
 'arp.hw.size',
 'icmp.checksum',
 'icmp.seq_le',
 'icmp.unused',
 'http.content_length',
 'http.response',
 'http.tls_port',
 'tcp.ack',
 'tcp.ack_raw',
 'tcp.checksum',
 'tcp.connection.fin',
 'tcp.connection.rst',
 'tcp.connection.syn',
 'tcp.connection.synack',
 'tcp.flags',
 'tcp.flags.ack',
 'tcp.len',
 'tcp.seq',
 'udp.stream',
 'udp.time_delta',
 'dns.qry.name',
 'dns.qry.qu',
 'dns.qry.type',
 'dns.retransmission',
 'dns.retransmit_request',
 'dns.retransmit_request_in',
 'mqtt.conflag.cleansess',
 'mqtt.conflags',
 'mqtt.hdrflags',
 'mqtt.len',
 'mqtt.msg_decoded_as',
 'mqtt.msgtype',
 'mqtt.proto_len',
 'mqtt.topic_len',
 'mqtt.ver',
 'mbtcp.len',
 'mbtcp.trans_id',
 'mbtcp.unit_id',
 'Attack_label',
 'http.request.method-0',
 'http.request.method-0.0',
 'http.request.method-GET',
 'http.request.method-OPTIONS',
 'http.request.method-POST',
 'http.request.method-PROPFIND',
 'http.request.method-PUT',
 'http.request.method-SEARCH',
 'http.request.method-TRACE',

In [68]:
empty_cols = [col for col in df.columns if df[col].isnull().all()]
empty_cols

[]

In [8]:
feat_cols = list(df.columns)
feat_cols.remove(label_col)
feat_cols

['arp.opcode',
 'arp.hw.size',
 'icmp.checksum',
 'icmp.seq_le',
 'icmp.unused',
 'http.content_length',
 'http.response',
 'http.tls_port',
 'tcp.ack',
 'tcp.ack_raw',
 'tcp.checksum',
 'tcp.connection.fin',
 'tcp.connection.rst',
 'tcp.connection.syn',
 'tcp.connection.synack',
 'tcp.flags',
 'tcp.flags.ack',
 'tcp.len',
 'tcp.seq',
 'udp.stream',
 'udp.time_delta',
 'dns.qry.name',
 'dns.qry.qu',
 'dns.qry.type',
 'dns.retransmission',
 'dns.retransmit_request',
 'dns.retransmit_request_in',
 'mqtt.conflag.cleansess',
 'mqtt.conflags',
 'mqtt.hdrflags',
 'mqtt.len',
 'mqtt.msg_decoded_as',
 'mqtt.msgtype',
 'mqtt.proto_len',
 'mqtt.topic_len',
 'mqtt.ver',
 'mbtcp.len',
 'mbtcp.trans_id',
 'mbtcp.unit_id',
 'Attack_label',
 'http.request.method-0',
 'http.request.method-0.0',
 'http.request.method-GET',
 'http.request.method-OPTIONS',
 'http.request.method-POST',
 'http.request.method-PROPFIND',
 'http.request.method-PUT',
 'http.request.method-SEARCH',
 'http.request.method-TRACE',

In [9]:
X = df.drop([label_col], axis=1)
y = df[label_col]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

In [11]:
label_encoder = LabelEncoder()
y_train =  label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [12]:
label_encoder.classes_

array(['DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 'Normal'],
      dtype=object)

In [13]:
min_max_scaler = MinMaxScaler()
X_train =  min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [14]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [15]:
input_shape = X_train.shape[1]
print(X_train.shape, X_test.shape)
print(input_shape)

(1239082, 96, 1) (413028, 96, 1)
96


In [16]:
num_classes = len(np.unique(y_train))
num_classes

5

In [17]:
Y_train = to_categorical(y_train, num_classes=num_classes)
Y_test = to_categorical(y_test, num_classes=num_classes)

In [18]:
# In order to ignore FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

K = keras.backend

input_shape = X_train.shape[1]

model = Sequential()
model.add(Convolution1D(filters=128, kernel_size=6, input_shape=(input_shape, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Convolution1D(filters=256, kernel_size=6))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Convolution1D(filters=256, kernel_size=6))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary() 

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

early_stop_callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=2, min_lr=0.0001)

with tf.device('/CPU:0'):
    X_train = tf.convert_to_tensor(X_train)
    Y_train = tf.convert_to_tensor(Y_train)
    X_test = tf.convert_to_tensor(X_test)
    Y_test = tf.convert_to_tensor(Y_test)

with tf.device('/GPU:0'):
    history = model.fit(X_train, Y_train,
                                batch_size=128,
                                epochs=30,
                                verbose=True, callbacks=[reduce_lr, early_stop_callback],
                                validation_data=(X_test, Y_test))  

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 91, 128)           896       
                                                                 
 batch_normalization (BatchN  (None, 91, 128)          512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 91, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 86, 256)           196864    
                                                                 
 batch_normalization_1 (Batc  (None, 86, 256)          1024      
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 86, 256)           0